In [3]:
import altair as alt
import numpy as np
import pandas as pd

from datetime import timedelta

In [4]:
chosen_owners = ["Eli", "Heather", "Kensie", "Sam", "Rachel"]
days = 150

In [5]:
# Load Data
df = pd.read_json("workitems.json")
df["iterationStartDate"] = pd.to_datetime(df["iterationStartDate"])
df["workStartDate"] = pd.to_datetime(df["workStartDate"])
df["finishDate"] = pd.to_datetime(df["finishDate"])

In [6]:
# Filter Data based on `chosen_owners` and `days`
filtered = df[df.owner.isin(chosen_owners)]

end_date = filtered["iterationStartDate"].max()
start_date = end_date - timedelta(days)

filtered = filtered[filtered.iterationStartDate.between(start_date, end_date)]

In [7]:
filtered = df

In [15]:
from collections import defaultdict
import json
# Wrangle data to get cumulative points over time
date_range = pd.date_range(start_date, end_date)
points_over_time = pd.pivot_table(filtered, values="points", index=["finishDate"], columns=["owner"], aggfunc=np.sum, fill_value=0)
points_over_time = points_over_time.reindex(date_range, fill_value=0)
points_over_time = points_over_time.cumsum()

d3_friendly = defaultdict(list)
for date, row in points_over_time.iterrows():
    for owner in chosen_owners:
        d3_friendly[owner].append((date.to_pydatetime().isoformat(),int(row[owner])))
    
d3_friendly
with open("pointsOverTime.json", "w") as output:
    json.dump(d3_friendly, output)